In [ ]:
# based on: https://www.kaggle.com/jutrera/training-a-densenet-for-the-stanford-car-dataset

In [47]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K

In [48]:
img_width, img_height = 224, 224
nb_train_samples = 512
n_classes = 10
epochs = 10
batch_size = 32

In [49]:
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    'out_keras',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 512 images belonging to 10 classes.


In [55]:
import urllib.request

# https://github.com/fchollet/deep-learning-models/releases/tag/v0.8
model_url = 'https://media-exp1.licdn.com/dms/image/C4E2CAQFdd_nVjxZAyw/comment-image-shrink_8192_1280/0?e=1603648800&v=beta&t=Ik_dpCkNbnboJr01ofO9HpEP4ESB4HAH18hcLt3-eG4'
model_name = 'densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5'

urllib.request.urlretrieve(model_url, model_name)

('densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5',
 <http.client.HTTPMessage at 0x1be1fbd3648>)

In [61]:
def build_model():
    base_model = densenet.DenseNet121(input_shape = (224, 224, 3),
                                      weights = 'densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                      include_top = False,
                                      pooling = 'avg')

    for layer in base_model.layers:
        layer.trainable = True

    x = base_model.output
    x = Dense(100, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    x = Dense(50, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

In [62]:
model = build_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'mse'])

In [63]:
model_history = model.fit_generator(train_generator, epochs = epochs)

Epoch 1/10
16/16 [==============================] - 817s 51s/step - loss: 154.0357 - acc: 0.0977 - mse: 0.0930
Epoch 2/10
16/16 [==============================] - 788s 49s/step - loss: 38.9923 - acc: 0.0938 - mse: 0.0900
Epoch 3/10
16/16 [==============================] - 784s 49s/step - loss: 34.2010 - acc: 0.0918 - mse: 0.0900
Epoch 4/10
16/16 [==============================] - 788s 49s/step - loss: 15522730.3113 - acc: 0.0898 - mse: 0.1016
Epoch 5/10
16/16 [==============================] - 801s 50s/step - loss: 33.5525 - acc: 0.0918 - mse: 0.0901
Epoch 6/10
16/16 [==============================] - 786s 49s/step - loss: 30.5643 - acc: 0.0977 - mse: 0.0901
Epoch 7/10
16/16 [==============================] - 795s 50s/step - loss: 29.9235 - acc: 0.1016 - mse: 0.0900
Epoch 8/10
16/16 [==============================] - 818s 51s/step - loss: 29.5563 - acc: 0.1035 - mse: 0.0900
Epoch 9/10
16/16 [==============================] - 817s 51s/step - loss: 29.2542 - acc: 0.0801 - mse: 0.0900
Epo

In [64]:
model.save('bmv_model.h5')

In [65]:
model.summary()

Model: "model_430"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_15 (ZeroPadding2 (None, 230, 230, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_15[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________

conv5_block7_0_relu (Activation (None, 7, 7, 704)    0           conv5_block7_0_bn[0][0]          
__________________________________________________________________________________________________
conv5_block7_1_conv (Conv2D)    (None, 7, 7, 128)    90112       conv5_block7_0_relu[0][0]        
__________________________________________________________________________________________________
conv5_block7_1_bn (BatchNormali (None, 7, 7, 128)    512         conv5_block7_1_conv[0][0]        
__________________________________________________________________________________________________
conv5_block7_1_relu (Activation (None, 7, 7, 128)    0           conv5_block7_1_bn[0][0]          
__________________________________________________________________________________________________
conv5_block7_2_conv (Conv2D)    (None, 7, 7, 32)     36864       conv5_block7_1_relu[0][0]        
__________________________________________________________________________________________________
conv5_bloc

In [66]:
# load model in tf
tf.keras.backend.set_learning_phase(0) 
tf_model = tf.keras.models.load_model('bmw_model.h5', custom_objects=None, 
                                      compile=True)

In [67]:
from tensorflow.python.framework import graph_io

# print input / output nodes
print([t.op.name for t in model.inputs], [t.op.name for t in model.outputs])

session = tf.keras.backend.get_session()
graph = session.graph

with graph.as_default():
    graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
    graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, [t.op.name for t in model.outputs])
    graph_io.write_graph(graphdef_frozen, '.', name='bmw_frozen_model.pb', as_text = False)

['input_8'] ['dense_1290/Softmax']
INFO:tensorflow:Froze 610 variables.
INFO:tensorflow:Converted 610 variables to const ops.
